In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sys
import re
from selenium.common.exceptions import TimeoutException
DRIVER_PATH = 'C:/Users/Public/chrome_driver/chromedriver.exe'

In [ ]:
class PersonObjectFV():
    def __init__(self):    
        self.EMAIL = 'John_doe@aol.com'
        self.FULL_NAME = "JOHN DOE"
        self.STREET = "MUSTERSTRAße 12"
        self.CITY = "MUSTERSTADT"
        self.ZIP = "6969"
        self.PHONE = "000"
        # TO DO nothing to do with a person self.COUPON = "POOR AMON DID NOT GET PAPUA"


In [ ]:
class FloavitroCheckoutValues():    
    def __init__(self):
        self.email = "/html/body/div[2]/div/div/div/div[1]/form[1]/fieldset/input"
        self.full_name = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/input[1]'
        self.street_address = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/input[2]'
        self.city = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/div/div[1]/input'
        self.country = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/select/option[11]'
        self.zip_code = "/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/div/div[3]/input"
        self.phone = '/html/body/div[2]/div/div/div/div[1]/form[3]/fieldset/input'
        self.coupon = "/html/body/div[2]/div/div/div/div[1]/form[4]/input"
        self.checkout_button = '/html/body/div[2]/div/div/div/div[2]/a'
    
    

In [ ]:
## clone and sex selectable
sex_label_with_clone = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div[1]/div[1]/label'
sex_clone_selection_male = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div[1]/div[2]/div[1]'
sex_clone_selection_female = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div[1]/div[2]/div[2]'
clone_secelection_label_with_sex =    '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div[2]/div[1]/label'
clone_list_with_sex = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div[2]/div[2]/*'
###
## clone selectable
clone_secelection_label_without_sex = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div/div[1]/label' # this one is for LABLE CLONE
clone_list_without_sex = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div/div[2]/*'

###
cart_element_clone_or_gender = "/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[2]/input[2]"
cart_element = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div/input[2]'


class Floravitro:
    
    def __init__(self):
        self.person = PersonObjectFV()
        self.checkout_values = FloavitroCheckoutValues()
        self.DRIVER_PATH = 'C:/Users/lavicola/Downloads/chromedriver_win32/chromedriver'
        self.WEBSITE = 'https://www.floravitro.com/shop'

    def start(self):
        self.driver = webdriver.Chrome(executable_path=self.DRIVER_PATH)
        self.driver.get(self.WEBSITE)
        #self.add_to_cart()
        #self.checkout()
    
        
    def press_add_to_cart_button(self):
        self.driver.find_element_by_xpath(cart_element_clone_or_gender).click()
    
    def press_add_to_cart_button2(self):
        self.driver.find_element_by_xpath(cart_element).click()
    
    
    
    # product has male and female    
    def isSexed(self):
        # check for sex label
        try:
            element = WebDriverWait(self.driver, 0.08).until(EC.presence_of_element_located((By.XPATH, sex_label_with_clone)))
        except:
            return False
        text = self.driver.find_element_by_xpath(sex_label_with_clone).text
        if(text != "Sex"):
            return False
        return True
    # product has a list of clones
    def isCloned(self,isSexed):
        if(isSexed):
            try:
                element = WebDriverWait(self.driver, 0.08).until(EC.presence_of_element_located((By.XPATH, clone_secelection_label_with_sex)))
            except:
                return False        
        else: 
            try:
                element = WebDriverWait(self.driver, 0.08).until(EC.presence_of_element_located((By.XPATH, clone_secelection_label_without_sex)))
            except:
                return False   
        if(isSexed):            
            text = self.driver.find_element_by_xpath(clone_secelection_label_with_sex).text
        else:
            text = self.driver.find_element_by_xpath(clone_secelection_label_without_sex).text
        if(text != "Clone"):
            return False        
        return True
        

        
    def add_to_cart(self,number=None,sex=None):
        #
        clone_selected = None
        sex_selected = None
        isSex = self.isSexed()
        isClone = self.isCloned(isSex)
        if(isClone):
            clones = self.get_clonelist(isSex)
            #select any clone
            for clone in clones:
                clone_selected = self.isCloneSelectable(isSex,clone)
                if(clone_selected):
                    break
        if(isSex):
            sex_selected = self.isSexSelectable("Female")
            if(not sex_selected):
                sex_selected = self.isSexSelectable("Male")
        if(clone_selected == True and (sex_selected == None or sex_selected == True)):
            self.press_add_to_cart_button()
            print("added!")
            return True
        if(clone_selected == None and sex_selected == None):
            self.press_add_to_cart_button2()
            print("added!")
            return True
        print("not added :/")
        return False
        
        
    def get_clonelist(self,isSexed):
        clones = []
        # {} starts with 1
        if(isSexed):
            div_element = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div[2]/div[2]/div[{}]' 
            values = self.driver.find_elements_by_xpath(clone_list_with_sex)
        else:
            div_element = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div/div[2]/div[{}]'
            values = self.driver.find_elements_by_xpath(clone_list_without_sex)
        for i in range(0,len(values)):
            clone_value = div_element.format(i+1)
            clone_number = self.driver.find_element_by_xpath(clone_value).text.replace("-","").strip("0")
            clones.append(clone_number)            
        return clones
    
    #Select Clone if possible and return True else return False
    def isCloneSelectable(self,isSexed,number):
        #aria-disabled
        # {} starts with 1
        if(isSexed):
            div_element = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div[2]/div[2]/div[{}]' 
            values = self.driver.find_elements_by_xpath(clone_list_with_sex)
        else:
            div_element = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div[1]/div/div[2]/div[{}]'
            values = self.driver.find_elements_by_xpath(clone_list_without_sex)
        for i in range(0,len(values)):
            clone_value = div_element.format(i+1)
            clone_number = self.driver.find_element_by_xpath(clone_value).text.replace("-","").strip("0")
            if(clone_number == number):                
                return_value = self.driver.find_element_by_xpath(clone_value).get_attribute("aria-disabled")
                if(return_value == "true"):
                    return False
                else:
                    self.driver.find_element_by_xpath(clone_value).click()
                    return True
        return False#not found ...

    def isSexSelectable(self,sex):
        if(sex == "Male"):
            div_element = sex_clone_selection_male
            return_value = self.driver.find_element_by_xpath(div_element).get_attribute("aria-disabled")
        else:
            div_element = sex_clone_selection_female
            return_value = self.driver.find_element_by_xpath(div_element).get_attribute("aria-disabled")
        if(return_value == "true"):            
            return False
        self.driver.find_element_by_xpath(div_element).click()
        return True
            
    def checkout(self):
        self.driver.get("https://www.floravitro.com/checkout") 
        try:
            element = WebDriverWait(self.driver, 0.08).until(EC.presence_of_element_located((By.XPATH, self.checkout_values.checkout_button)))
        except: #could not click element for some reason ....
            return False                            
        self.driver.find_element_by_xpath(self.checkout_values.country).click()
        self.driver.find_element_by_xpath(self.checkout_values.email).send_keys(self.person.EMAIL)
        self.driver.find_element_by_xpath(self.checkout_values.city).send_keys(self.person.CITY)
        self.driver.find_element_by_xpath(self.checkout_values.full_name).send_keys(self.person.FULL_NAME)
        self.driver.find_element_by_xpath(self.checkout_values.street_address).send_keys(self.person.STREET)
        self.driver.find_element_by_xpath(self.checkout_values.zip_code).send_keys(self.person.ZIP)
        self.driver.find_element_by_xpath(self.checkout_values.phone).send_keys(self.person.PHONE)
        
    
    
    

In [ ]:
fv = Floravitro()


In [ ]:
fv.start()

In [ ]:
links =[
    "https://www.floravitro.com/product/nepenthes-bokorensis-mt-bokor-cambodia",
    "https://www.floravitro.com/product/nepenthes-sp-harau-harau-valley-sumatra",
    "https://www.floravitro.com/product/nepenthes-veitchii-mt-murud-striped-x-candy-striped",
    "https://www.floravitro.com/product/nepenthes-veitchii-mt-murud-striped-x-green-peristome"
    
]

In [ ]:
for link in links:    
    fv.driver.get(link)
    fv.add_to_cart()
    time.sleep(0.2)
fv.checkout()




# AW

In [ ]:
class PersonObjectAW:
    def __init__(self):    
        self.EMAIL = ''
        self.PASSWORD = ""

        
class AWValues:    
    def __init__(self):
        self.email = "/html/body/div[2]/div/div/div/div[1]/form[1]/fieldset/input"
        self.full_name = '//*[@id="name"]'
        self.street_address = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/input[2]'
        self.city = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/div/div[1]'
        self.country = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/select/option[11]'
        self.zip_code = '/html/body/div[2]/div/div/div/div[1]/form[2]/fieldset/div/div[3]/input'
        self.phone = '/html/body/div[2]/div/div/div/div[1]/form[3]/fieldset/input'
        self.coupon = "/html/body/div[2]/div/div/div/div[1]/form[4]/input"
        self.checkout_button = '/html/body/div[2]/div/div/div/div[2]/a'
    

class PlantObject:
    def __init__(self,name="",clone="",size="",link=None,price=100):
        self.name = name
        self.clone = clone
        self.size = size
        self.link = link
        self.price = price
        self.available = False
    
    def setName(self,name):
        self.name = name
    
    def setClone(self,clone):
        self.clone=clone
    
    def setSize(self,size):
        self.size = size
        
    def setLink(self,link):
        self.link = link
        
    def setPrice(self,price):
        self.price = price
    
    def setAvailable(self):
        self.available = True
    
    
    


In [ ]:
## clone and sex selectable
clone_label = '//*/form/table[2]/tbody/tr[1]/td[2]/select/*'
size_label = '//*/form/table[2]/tbody/tr[2]/td[2]/select/*'
available_text = "availability_text"
###
iframe_path = '/html/body/div[4]/div[1]/div[1]/div[1]/div[3]/iframe'
link_paths = "/html/body/div/table/tbody/tr/td/a['href']"
###
cart_element = '/html/body/div[3]/div[1]/div[1]/div[2]/div[5]/form/div/input[2]'
###
proceed_to_checkout = "/html/body/div[4]/div[1]/div[1]/div[2]/form[2]/table/tbody/tr[2]/td[2]/input"
checkout_name = '//*/form/table[1]/tbody[2]/tr[2]/td[3]/input'
checkout_password = '//*/form/table[1]/tbody[2]/tr[3]/td[3]/input'
continue_button = '/html/body/div[4]/div[1]/div[1]/div[2]/form/table[2]/tbody/tr/td/input'
continue_button2 = "//*/form/table[2]/tbody/tr/td/input"
continue_button2 = "//*/form/table[3]/tbody/tr/td[2]/input"

class Wistuba:    
    def __init__(self,targets):
        self.person = PersonObjectAW()
        self.checkout_values = AWValues()
        self.DRIVER_PATH = 'C:/Users/lavicola/Downloads/chromedriver_win32/chromedriver'
        self.WEBSITE = 'https://www.wistuba.com/inventory/index.php'
        self.driver = webdriver.Chrome(executable_path=self.DRIVER_PATH)
        self.driver.get(self.WEBSITE)
        self.nepenthes_list = self.GetAllNepenthesNames()
        self.plant_objects = targets
        self.pattern = re.compile("male|female")
            
    def GetAllNepenthesNames(self):
        """
        returns a list of every nepenthes
        """
        f = open("nepenthes_species.csv", "r")
        plants = f.read().splitlines()
        return plants
    
    
    def CleanString(self,product_name):
        """
        Argument product_name: The Name of a Plant with possible random strings e.g : Nepenthes attenboroughii (Mt. Victoria, Palawan) 
        We remove everything which has nothing to do with the Plant name. The return value is attenboroughii
        """
        clean_name = []
        #remove every word which has nothing to do with a species of a Nepenthes
        product_name = product_name.replace("Nepenthes","").replace("(","").replace(")","")
        # only random words like the location/mountain and nepenthes names are left. we will now remove every random word
        for word in product_name.split(" "):
            for nepenthes_name in self.nepenthes_list:
                if(word == nepenthes_name ):
                    clean_name.append(word)
        return clean_name                
    
    def IsSamePlant(self,product_plant,target_plant):
        '''
        Arguments:
        product_plant is a list of a cleaned String with only Nepenthes Names The Plant The Shop is offering!
        target_plant is a list of a cleaned String with only Nepenthes Names. The Plant The Customer wants!
        If the lists are the SAME AND in the same order of the plant we Customer is looking and have the same length we assume they are equal 
        '''        
        if(len(target_plant) != len(product_plant)):
            return False        
        for i in range(0,len(target_plant)):
            if(product_plant[i] != target_plant[i]):
                return False        
        return True    
    
    def CheckIfATargetNepenthesIsAvailable(self,min_amount=0):
        """
        This function is suppossed to get called every X Minutes. It will check if a Plant we are looking for is available
        and if so it calls the proccess to add to cart / checkout
        """        
        data = wistuba.GetInventory() # the latest inventory data
        links = []
        available = 0
        for plant in self.plant_objects:
            for inventory_plant in data:
                if(wistuba.IsSamePlant(wistuba.CleanString(inventory_plant[0]),wistuba.CleanString(plant.name))):
                    plant.setLink(inventory_plant[1])
                    plant.setAvailable()
                    available +=1                    
                    break                    
        
        if(available>=min_amount):
            self.start(self.plant_objects)        
    
    def start(self,plant_objects):
        '''
        Argument: links is a list of every link to a certain product page
        Proccess: adds every item to the shop and will than checkout
        '''
        for plant in plant_objects:
            if(plant.available):
                self.ChangeProductSite(plant.link)
                if(self.ChoseClone(plant)):
                    if(self.isPriceOkay(plant.price)):
                        self.press_add_to_cart_button() 
        self.Checkout()
        return
    
    def press_add_to_cart_button(self):
        self.driver.find_element(By.ID,"submitbutton").click()
    
    def ChangeProductSite(self,link):
        self.driver.get(link)

    def AddToCart(self):
        """
        Check if a Nepenthes(clone--> size) is Available if so add to cart        
        """
        
        if(self.ChoseClone()):
            self.press_add_to_cart_button()
            return True
        return False
    
    # get sizes of products    
    def ChoseSize(self,size=""):
        # chose a size
        elements = self.driver.find_elements_by_xpath(size_label)
        if(size == ""):
            for element in elements:
                element.click()
                if(self.isAvailable()):                    
                    return True
        else:
            for element in elements:
                if(element.text == size):
                    element.click()
                    if(self.isAvailable()):                        
                        return True
        return False            
            
    def ChoseClone(self,plant_object):
        """
        argument clone: the clone the Customer wants
        We check if a clone is available if none was given we just check if one clone is 
        available the combine that with checking every size        
        
        """
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, clone_label)))#wait for the page to be loaded
        elements = self.driver.find_elements_by_xpath(clone_label)        
        if(plant_object.clone == ""):
            for element in elements:
                element.click()
                if(self.ChoseSize(plant_object.size)):                    
                    return True
        else:
            for element in elements:
                if(self.GetClone(element.text,plant_object.clone) == plant_object.clone):
                    element.click()
                    if(self.ChoseSize(plant_object.size)):
                        return True
        return False
            
            
        return False # nothing was available
                
    def Checkout(self):
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[4]/div[2]/div[2]/a[2]"))).click()
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, checkout_name))).send_keys(self.person.EMAIL)
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, checkout_password))).send_keys(self.person.PASSWORD)
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, continue_button))).click()
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, continue_button))).click()
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, continue_button2))).click()


    def GoToIFrame(self):
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH,iframe_path)))
        iframe = WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH,iframe_path)))
        self.driver.switch_to.frame(iframe)
        
    def GetInventory(self):
        inventory = []
        self.GoToIFrame()
        data = wistuba.driver.find_elements_by_xpath(link_paths)
        for i in data:
            if(i.get_attribute("href") != None): # for some reason tuple are empty
                inventory.append((i.text,i.get_attribute("href")))    
        self.driver.switch_to.default_content()
        return inventory
    
    def CheckProductInventoryList(self,inventory_tuple_list,target_plants):
        target_links = []
        for inventory_tuple in inventory_tuple_list:
            for target_plant in target_plants:
                if(target_plant in inventory_tuple[0]):
                    target_links.append(inventory_tuple[1])
        return target_links
    
        
    def GetClone(self,plant,clone):
        """
        Every entry has useless letters e.g "AW_N_klossi03" we just need the number
        OR we need the gender of the plant e.g "AW_N_clip-u(male)" --> male
        """
        #we get both cases female or male we just gotta extract the whole word now
        if("male" in clone):
            results = self.pattern.findall(plant)
            if(len(results)<= 0 ):                
                return None #Gender does not exist for this plant-
            else:
                return results[0]
        else:    
            return ''.join(filter(str.isdigit, plant)).strip("0")
        
            
    
    def isAvailable(self):
        element = self.driver.find_element(By.ID,"availability_text")
        # Not available within the coming months
        if("Not" in element.text):
            return False
        else:
            # Available within the coming months
            return True
        
    def isPriceOkay(self,price):
        element = self.driver.find_element(By.ID,"price")
        product_price = element.text.replace(",",".").replace("EUR","")
        if(float(price) <= float(product_price)):
            return True
        else:
            return False
        
    
    
    

In [ ]:
#target_plants = ["erucoides","hamata x edwardsiana","klossii","klossii","klossii"]
target_plants = ["robcantleyi-x-lowii-x-truncata"]
target_size = [""]
target_clones = [""]
target_price = [999]
plants = []

for i in range(0,len(target_plants)):
    plants.append(PlantObject(target_plants[i],clone=target_clones[i],size=target_size[i],price=target_price[i]))



In [ ]:
wistuba = Wistuba(plants)


In [ ]:
wistuba.CheckIfATargetNepenthesIsAvailable()

## taerwa Tropicals


In [168]:
class Laurent():
        
    
    def __init__(self):
        self.DRIVER_PATH = DRIVER_PATH
        self.WEBSITE = 'https://www.taerwetropicals.com/shop/' #https://www.taerwetropicals.com/product-category/carnivorous-plants/exotica_plants/ and https://www.taerwetropicals.com/product-category/carnivorous-plants/taerwe_tropicals/ possible 
        return
        
    def start(self):
        self.driver = webdriver.Chrome(executable_path=self.DRIVER_PATH)
        self.driver.get(self.WEBSITE)
        highest_page = self.getNumberofPages()
        #get every product and add every product to the basket
        for i in range(2,highest_page):      
            products = self.getProductPages()
            for product in products:
                self.switchToProductPage(product)
                self.addBasket()
            next_page = "https://www.taerwetropicals.com/shop/page/"+str(i)+ "/"
            self.nextPage(next_page)
        #self.add_to_cart()
        #self.checkout()
        return
        
    
    def changeDetailPage(self):
        return
    
    
    def switchToProductPage(self,link):
        self.driver.get(link)                
        return
        
    
    
    def addBasket(self):
        try:
            element = WebDriverWait(self.driver, 1).until(EC.presence_of_element_located((By.XPATH,"//button[@class='single_add_to_cart_button button alt']")))
            element.click()
        except TimeoutException as ex:
            return
        return
            
    
    def viewCart(self):
        self.driver.get("https://www.taerwetropicals.com/cart/")        
        return
    
    
    
    def nextPage(self,link):
        self.driver.get(link)
        #self.driver.find_element_by_xpath("//a[contains(text(),'→')]").click()
        return
    
    def getNumberofPages(self):
        pages = self.driver.find_elements_by_xpath("//a[@class='page-numbers']")
        highest_number = len(pages)
        return int(pages[highest_number-1].text)+1

        
    
    
    
    def getProductPages(self):
        nodes = test.driver.find_elements_by_xpath("//*[@class='products columns-3']/li")
        products = test.driver.find_elements_by_xpath("//*[@class='mkdf-product-list-title']/a")
        valid_products = []
        for i in range(0,len(nodes)):
            if( not "sold" in nodes[i].text.lower()):
                if(products[i].get_attribute("href") != None): # check if href exist
                    valid_products.append(products[i].get_attribute("href"))
        return valid_products
        
""" dumb solution dont check for sold    
        products = self.driver.find_elements_by_xpath("//*[@class='mkdf-product-list-title']/a")
        valid_products = [] # make sure to return only products with a link
        for product in products:
            if(product.get_attribute("href") != None): # for some reason tuple are empty
                valid_products.append(product.get_attribute("href"))                                
        return valid_products
"""

    
    
    
    
    
    
    
    
    
    

' dumb solution dont check for sold    \n        products = self.driver.find_elements_by_xpath("//*[@class=\'mkdf-product-list-title\']/a")\n        valid_products = [] # make sure to return only products with a link\n        for product in products:\n            if(product.get_attribute("href") != None): # for some reason tuple are empty\n                valid_products.append(product.get_attribute("href"))                                \n        return valid_products\n'

In [169]:
test = Laurent()

In [170]:
test.start()

In [171]:
links = test.getProductPages()

In [115]:
test.driver.find_element_by_xpath("//button[@class='single_add_to_cart_button button alt']").click()

In [68]:
for i in a:
    print(i.text)
    if(i.get_attribute("href") != None): # for some reason tuple are empty
        print("found")


In [144]:
nodes[0].text.lower()

'sold\nn. (rokko ‘exotica’ x boschiana) x (veitchii (m) x burbidgeae) – specimen plant\n€125,00'